## Capstone Project

In [ ]:
# install packages
!pip install pycaret
!pip install unidecode

In [ ]:
# download corpus of stop words
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# import modules
import numpy as np
import pandas as pd
import plotly.express as px
import scipy.stats as ss
import unidecode
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
# load the data
data = pd.read_csv("pet food data.csv")

There are 684 entries that do not have price data, 548 entries that do not have an ingredients list, 755 entries that do not have a total package size, and 747 entries that do not have a unit package size.

In [ ]:
# get a description of the unprocessed data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8754 entries, 0 to 8753
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  8754 non-null   object 
 1   category              8754 non-null   object 
 2   sub_category          8754 non-null   object 
 3   product               8754 non-null   object 
 4   variant               8754 non-null   object 
 5   description           8754 non-null   object 
 6   ingredients_stdlist   8206 non-null   object 
 7   price_usd             8070 non-null   float64
 8   company_parent        8754 non-null   object 
 9   company               8754 non-null   object 
 10  brand                 8754 non-null   object 
 11  total_pack_size_ml_g  7999 non-null   float64
 12  unit_pack_size_ml_g   8007 non-null   float64
dtypes: float64(3), object(10)
memory usage: 889.2+ KB


In [ ]:
# determine if there are any duplicate rows
np.sum(data.duplicated())

0

### Initial Cleaning for Visualization

In [ ]:
# drop the 900 entries that do not have price data or an ingredients list
data.dropna(inplace=True, subset=["price_usd", "ingredients_stdlist"])

# remove 1 outlier price with invalid pack size
data = data[data.price_usd < 50]

# drop 32 therapeutic supplements 
data = data.loc[~data.ingredients_stdlist.str.startswith("active"), :]

# reset the index
data.reset_index(drop=True, inplace=True)

In [ ]:
# transform the string dates into datetime objects and create a year column
data.loc[:, "date"] = pd.to_datetime(data.date.str[:10])
data["year"] = data.date.apply(lambda x: x.year)

In [ ]:
# drop the "category" column since it contains one value "Pet Food" and
# the "variant" column since it is equivalent to the "product" column
data.drop(columns=["category", "product"], inplace=True)

In [ ]:
# transform the "sub_category" column into separate columns and drop "sub_category"
data["pet_type"] = data.sub_category.apply(lambda x: "cat" if ("Cat" in x) else "dog")
data["meal_type"] = data.sub_category.apply(lambda x: "primary" if ("Food" in x) else "treats")
data["food_type"] = data.sub_category.apply(lambda x: "dry" if ("Dry" in x) else "wet")
data.drop(columns="sub_category", inplace=True)

In [ ]:
# create a function to replace dicritic characters with ASCII characters
def replace_dicritic(string):
    return unidecode.unidecode(string)

In [ ]:
# replace the dicritic characters in "company_parent", "company", "brand",
# "description", and "ingredients_stdlist"
data.loc[:, "company_parent"] = data.company_parent.apply(replace_dicritic)
data.loc[:, "company"] = data.company.apply(replace_dicritic)
data.loc[:, "brand"] = data.brand.apply(replace_dicritic)
data.loc[:, "description"] = data.description.apply(replace_dicritic)
data.loc[:, "ingredients_stdlist"] = data.ingredients_stdlist.apply(replace_dicritic)

In [ ]:
# get a description of the cleaned data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7822 entries, 0 to 7821
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  7822 non-null   datetime64[ns]
 1   variant               7822 non-null   object        
 2   description           7822 non-null   object        
 3   ingredients_stdlist   7822 non-null   object        
 4   price_usd             7822 non-null   float64       
 5   company_parent        7822 non-null   object        
 6   company               7822 non-null   object        
 7   brand                 7822 non-null   object        
 8   total_pack_size_ml_g  7526 non-null   float64       
 9   unit_pack_size_ml_g   7534 non-null   float64       
 10  year                  7822 non-null   int64         
 11  pet_type              7822 non-null   object        
 12  meal_type             7822 non-null   object        
 13  food_type         

### Visualizations

In [ ]:
# get the observations for Colgate only
colgate_only = data[data.company_parent.apply(lambda x: "colgate" in x.lower())]

In [ ]:
# perform a Box-Cox transformation on price_usd
box_cox = ss.boxcox(data.price_usd)

Boxplots

In [ ]:
# boxplot for cat vs dog
fig1 = px.box(data, x="pet_type", y="price_usd")
fig2 = px.box(colgate_only, x="pet_type", y="price_usd", category_orders={"pet_type": ["cat", "dog"]})

# boxplot for primary food vs treats
fig3 = px.box(data, x="meal_type", y="price_usd")
fig4 = px.box(colgate_only, x="meal_type", y="price_usd")

# boxplot for dry vs wet food
fig5 = px.box(data, x="food_type", y="price_usd")
fig6 = px.box(colgate_only, x="food_type", y="price_usd", category_orders={"food_type": ["dry", "wet"]})

# boxplot for year
fig7 = px.box(data, x="year", y="price_usd")

Histograms

In [ ]:
# histogram of price
fig8 = px.histogram(data.price_usd)

# histogram of price with Box-Cox transformation
fig9 = px.histogram(box_cox[0], nbins=50)

Scatter Plots

In [ ]:
# scatter plot of price vs unit pack size
fig10 = px.scatter(data.loc[~data.unit_pack_size_ml_g.isna(), :], x="unit_pack_size_ml_g", y="price_usd", trendline="ols", trendline_color_override="orange", range_y=[-2, 37])

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



### Text Transformations

In [ ]:
# get the first word in every ingredient list
first_ingredient = data.ingredients_stdlist.str.split(",").apply(lambda x: x[0])
first_ingredient = first_ingredient.str.split(" ").apply(lambda x: x[0])
first_ingredient = first_ingredient.str.split(":").apply(lambda x: x[0].lower())

In [ ]:
# look at the top 10 words by frequency count
first_ingredient.value_counts()[:10]

chicken    1890
wheat       692
beef        611
corn        420
water       351
turkey      291
salmon      249
lamb        228
rice        210
cowhide     159
Name: ingredients_stdlist, dtype: int64

In [ ]:
# create a function that filters the first_ingredient Series to
# 5 possible words
def first_word(string):
    if string == "chicken" or string == "wheat" or string == "beef" or string == "corn":
        return string
    else:
        return "other"

In [ ]:
# add a first_ingredient column to the dataset
data["first_ingredient"] =  first_ingredient.apply(first_word)

TFIDF

During testing, we discovered that using a TFIDF matrix with more than ~400 features in our regression models caused instability, resulting in errors that were $10^6$ or greater. To improve performance, we set max_features=400 in our TfidfVectorizer().

In [ ]:
# instantiate the TFIDF vectorizer
vectorizer = TfidfVectorizer(stop_words=stopwords.words("english"), max_features=400)

# transform the product descriptions in a TFIDF matrix
vectors = vectorizer.fit_transform(data.description)

# convert the TFIDF matrix into a dataframe
tfidf = pd.DataFrame(vectors.todense().tolist(), columns=vectorizer.get_feature_names())

Convert categorical variables to one-hot vectors

In [ ]:
# get all of the categorical variables in one dataframe
cat_vars = data[["pet_type", "meal_type", "food_type", "first_ingredient"]]

# transform the categorical variables into one-hot vectors
one_hot = pd.get_dummies(cat_vars, prefix="category")

### Combine all of the engineered features and create training and testing sets

In [ ]:
# combine the 1hot dataframe with the TFIDF dataframe
X = pd.concat([one_hot, tfidf], axis=1)

# create a target variable for the Box-Cox transformed prices
y = box_cox[0]

# create stratified training and testing sets
# prices box-cox
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=data.pet_type, random_state=38)

# original prices
X_train_org, X_test_org, y_train_org, y_test_org = train_test_split(X, data.price_usd, train_size=0.8, stratify=data.pet_type, random_state=38)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6257, 411), (1565, 411), (6257,), (1565,))

In [ ]:
X_train_org.shape, X_test_org.shape, y_train_org.shape, y_test_org.shape

((6257, 411), (1565, 411), (6257,), (1565,))

### pycaret test

In [ ]:
from pycaret.regression import *

In [ ]:
data2 = X.copy()

In [ ]:
data2["price"] = box_cox[0]

In [ ]:
exp_reg101 = setup(data = data2, target="price", data_split_stratify=["category_dog"], use_gpu=True)

,Description,Value
0,session_id,8110
1,Target,price
2,Original Data,"(7822, 412)"
3,Missing Values,False
4,Numeric Features,411
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(5475, 408)"


In [ ]:
best = compare_models(fold=5, sort="RMSE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.5500,0.5863,0.7656,0.6838,0.2809,1.3619,16.996
lightgbm,Light Gradient Boosting Machine,0.5865,0.6076,0.7792,0.6721,0.2896,1.4345,1.392
catboost,CatBoost Regressor,0.5973,0.6236,0.7896,0.6637,0.2933,1.5118,75.118
rf,Random Forest Regressor,0.5893,0.6313,0.7944,0.6595,0.2920,1.4211,20.164
xgboost,Extreme Gradient Boosting,0.6126,0.6653,0.8155,0.6410,0.2996,1.5925,1.420
gbr,Gradient Boosting Regressor,0.6408,0.6869,0.8286,0.6296,0.3064,1.6022,5.782
br,Bayesian Ridge,0.6606,0.7448,0.8629,0.5982,0.3165,1.6347,0.390
ridge,Ridge Regression,0.6631,0.7490,0.8654,0.5959,0.3172,1.6452,0.056
lr,Linear Regression,0.6749,0.7709,0.8778,0.5840,0.3202,1.6965,0.142
huber,Huber Regressor,0.6605,0.7749,0.8801,0.5819,0.3203,1.5914,1.376
